In [12]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



Los clasificadores Naive Bayes (Naive Bayes Classifier - NBC) se usan , como su nombre indica, para problemas de clasificación, y en concreto se pueden aplicar para texto.

En este ejemplo vamos a implementar un modelo NBC a un dataset de un portal de Noticias muy famoso en España. Cada usuario comparte un link a una noticia y le puede asignar una categoría.

In [3]:
noticias = pd.read_csv("./data/noticias.csv")
noticias.head()

,descripcion,categoria
0,"Aunque parezca mentira, las emisiones de dióxi...",cultura
1,Hubo un proyecto impulsado por la Unión Europe...,cultura
2,China ha confirmado la conclusión con éxito de...,tecnología
3,"En su fructífera carrera como humorista, actor...",cultura
4,Tras dos años de negociación entre la instituc...,cultura


La variable objetivo es categoria y la variable independiente es descripcion que contiene la descripcion de la noticia

In [4]:
noticias.categoria.value_counts()

cultura       9001
tecnología    4198
ocio          3296
Name: categoria, dtype: int64


Vemos que hay noticias de 3 tipos de categorías distintas.

Los clasificadores Naive Bayes esperan como input un vector, así que para poder entrenarlos tenemos que vectorizar el texto. Para ello una buena opción es usar vectorización Tf-IDF


### Eliminar Stopwords

Stopwords (palabras vacías) son palabras que no tienen ningún contenido semántico. Por ejemplo, en la frase el perro ladra el artículo el no aporta ningún valor a la frase.

In [6]:
with open("data/stopwords-es.json") as fname:
    stopwords_es = json.load(fname)

### Eliminar acentos

También vamos a eliminar los acentos de las palabras, ésto tiene una ventaja, y es que si en el conjunto de datos no tenemos confianza en la calidad de los escritores, al eliminar los acentos evitamos que si un escritor no usa acentos no considere sus palabras como palabras distintas.

En castellano, esto tiene un problema, y es que hay palabras con significado distinto que sólo se diferencian por la existencia de una tilde (se llaman palabras con acento diacrítico (por ejemplo, de y dé). Asumimos pues que el impacto de estas palabras no es muy alto.

In [7]:
vectorizador = TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es)

In [10]:
noticias.shape

(16495, 2)

In [11]:
vectorizador.fit_transform(noticias.descripcion)

C:\Users\mario\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
  warnings.warn('Your stop_word

<16495x59952 sparse matrix of type '<class 'numpy.float64'>'
	with 397698 stored elements in Compressed Sparse Row format>

Dicha matriz nos indica que tenemos 16495 artículos que tienen 59969 palabras distintas (sin contar acentos o stopwords)


Dado que los vectorizadores devuelven una matriz sparse (escasa) creamos un transformador que las convierta a densas

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

from scipy.sparse import issparse


# http://rasbt.github.io/mlxtend/
class DenseTransformer(BaseEstimator):
    def __init__(self, return_copy=True):
        self.return_copy = return_copy
        self.is_fitted = False

    def transform(self, X, y=None):
        if issparse(X):
            return X.toarray()
        elif self.return_copy:
            return X.copy()
        else:
            return X

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

In [14]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

Scikit-learn tiene tres implementaciones del clasificador Naive Bayes, GaussianNB, BernoulliNB y MultinomialNB, y cada una se diferencia por como calcula las probabilidades de que cada elemento aparezca en los datos.

GaussianNB asume que los datos siguen una distribución Gausiana

In [15]:
pipeline_gaussiano = make_pipeline(
    vectorizador,
    DenseTransformer(),
    GaussianNB()
)

In [16]:
pipeline_gaussiano.fit(X=noticias.descripcion, y=noticias.categoria)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(stop_words=['0', '1', '2', '3', '4', '5', '6',
                                             '7', '8', '9', '_', 'a',
                                             'actualmente', 'acuerdo',
                                             'adelante', 'ademas', 'ademÃ¡s',
                                             'adrede', 'afirmÃ³', 'agregÃ³',
                                             'ahi', 'ahora', 'ahÃ\xad', 'al',
                                             'algo', 'alguna', 'algunas',
                                             'alguno', 'algunos', 'algÃºn', ...],
                                 strip_accents='unicode')),
                ('densetransformer', DenseTransformer()),
                ('gaussiannb', GaussianNB())])

In [17]:
pipeline_gaussiano.predict(noticias.descripcion)

array(['cultura', 'cultura', 'tecnología', ..., 'cultura', 'tecnología',
       'ocio'], dtype='<U10')

In [18]:
from sklearn.metrics import f1_score


def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return f1_score(y, preds, average="micro")

In [19]:
cross_val_score(pipeline_gaussiano, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta)

C:\Users\mario\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
  warnings.warn('Your stop_word

array([0.62988784, 0.64352834, 0.64807517, 0.63776902, 0.63504092])

Vemos que el modelo funciona de igual forma ahora pese a que los datos de entrenamiento son bastante más pequeños (una matriz de dimensión 16495x500 en vez de una de dimensión 16495x59969

Las dos implementaciones de clasificadores NB más utilizadas para clasificación de texto son MultinomialNB (que asume que la distribución de probabilidades de las palabras en el conjunto de datos sigue una distribución multinomial y BernouilliNB , que asume que siguen una distribución de Bernouilli multivariable (donde la existencia de cada palabra se considera que es una variable binaria distinta).

Según la documentación, el clasificador MultinomialNB funciona bien con vectores TFIDF

In [20]:
pipeline_multinomial = make_pipeline(
    TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es, max_features=500),
    DenseTransformer(),
    MultinomialNB(),
)

In [21]:
cross_val_score(pipeline_multinomial, noticias.descripcion, noticias.categoria,
                scoring=f1_multietiqueta)

C:\Users\mario\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
  warnings.warn('Your stop_word

array([0.68717793, 0.67596241, 0.67202182, 0.68384359, 0.66656563])

Vemos que efectivamente, MultinomialNB parece funcionar mejor que GaussianNB para vectores TF-IDF.

Para el clasificador BernouilliNB se necesita tener los vectores de palabras cono vectores binarios (1 si la palabra existe o 0 si no), así que en vez de usar TfidfVectorizer en este caso usaremos el CountVectorizer pasandole el parámetro binary=True para que devuelva 1 ó 0 en vez del número de veces que aparece la palabra en la frase.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizador_count = CountVectorizer(stop_words=stopwords_es, binary=True, 
                                     strip_accents="unicode", max_features=1000)

In [23]:
vectorizador_count.fit(noticias.descripcion)
vectorizador_count.vocabulary_

C:\Users\mario\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
  warnings.warn('Your stop_word

{'tierra': 929,
 'verde': 969,
 '30': 25,
 'anos': 76,
 'epoca': 339,
 'principal': 767,
 'estudio': 369,
 'internacional': 502,
 'publicado': 796,
 'pasar': 709,
 'proyecto': 789,
 'union': 950,
 'europea': 373,
 'metodo': 600,
 'lengua': 529,
 'espanol': 351,
 'sistema': 882,
 'espanoles': 353,
 'puedan': 802,
 'entender': 334,
 'china': 167,
 'confirmado': 206,
 'exito': 381,
 'forma': 405,
 'totalmente': 935,
 'robot': 845,
 'logro': 554,
 'personal': 726,
 'humano': 464,
 'carrera': 157,
 'actor': 48,
 'director': 289,
 'escritor': 346,
 'decenas': 257,
 'peliculas': 712,
 'importantes': 478,
 'historia': 455,
 'cine': 176,
 'humor': 466,
 'mayoria': 586,
 'encuentran': 328,
 'publico': 799,
 'online': 678,
 'mejores': 592,
 'museo': 631,
 'finalmente': 400,
 'siglo': 874,
 'musica': 632,
 'texto': 926,
 'creado': 235,
 'nombre': 652,
 'obra': 664,
 'video': 976,
 'hombre': 458,
 'comida': 192,
 'muestra': 625,
 'problema': 773,
 'usando': 953,
 'google': 434,
 'dando': 250,
 'mun

In [24]:
pipeline_bernouilli = make_pipeline(
    vectorizador_count,
    DenseTransformer(),
    BernoulliNB(),
)

In [25]:
cross_val_score(pipeline_bernouilli, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta)

C:\Users\mario\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
  warnings.warn('Your stop_word

array([0.67262807, 0.68293422, 0.69051228, 0.6880873 , 0.67293119])